In [1]:
import RevPrep

In [2]:
import pandas as pd

In [3]:
import pickle as pkl
import glob
import numpy as np

In [4]:
#Using Bookbrowse
file = glob.glob("*.csv")[0]
df = pd.read_csv(file, delimiter=";", encoding="latin-1")
summaries = list(df["Summary"])

IndexError: list index out of range

In [5]:
#Using Goodreads
summaries = pkl.load(open("window_ds_big.pkl", "rb"))
tokenizer_ds = pkl.load(open("l_sums_filtered_grammar.pkl","rb"))

In [6]:
from RevPrep.Helpers.helpers import text_to_sentences
sen_counts = [len(text_to_sentences(i)) for i in summaries]

In [7]:
from tensorflow import keras as k
import numpy as np
tokenizertf = k.preprocessing.text.Tokenizer(20000)
tokenizertf.fit_on_texts(tokenizer_ds)

In [8]:
def is_good(model_name, sums, tokenizer, starti=6, rangei=26, inc=4, batch_size=8):
    from RevPrep.Generation.generate.base import generate 
    from RevPrep.Evaluations.scoring import analyse_sentiment, cal_generation_quality
    from RevPrep.Generation.generate.base import get_model_tokenizer, del_model_tokenizer
    from RevPrep.plotting import plot
    from tqdm.notebook import tqdm
    import numpy as np
    good_configs = []
    bad_configs = []
    model,tokenizer = get_model_tokenizer(model_name) 
    for i in tqdm(range(starti,rangei,inc), position=0, desc=f"Up to {rangei} windows, start: 6"):
        for i2 in tqdm([0,1,2,int(np.min([i/2, 5]))], desc=f"Up to {int(np.min([i/2, 5]))} overlap, start: 0, w: {i}", leave=False):
            good_method = False
            results_window = generate(model_name, sums, model, tokenizer, True, i,i2, batch_size=batch_size)
            c_rate_mean = results_window['compression_rate_mean']
            c_rate_std_dev = results_window['compression_rate_std_dev']
            gen_qual = cal_generation_quality(results_window["generated summaries"], results_window["source_texts"], tokenizertf)
            del results_window
        
            if(gen_qual["mean"] > 0.40 and c_rate_mean > 0.40 and c_rate_mean < 0.55):
                good_method = True
        
            if good_method:
                good_configs.append({"model_name": model_name, "window_size": i, "overlap": i2, "compression_rate mean": c_rate_mean, "compression_rate std dev": c_rate_std_dev, "gen_quality mean": gen_qual["mean"], "gen_quality std dev": gen_qual["std dev"], "example_gen": gen_qual["example_gen"], "example_src": gen_qual["example_source"]})
            else:
                bad_configs.append({"model_name": model_name, "window_size": i, "overlap": i2, "compression_rate mean": c_rate_mean, "compression_rate std dev": c_rate_std_dev, "gen_quality mean": gen_qual["mean"], "gen_quality std dev": gen_qual["std dev"], "example_gen": gen_qual["example_gen"], "example_src": gen_qual["example_source"]})
    del_model_tokenizer(model, tokenizer)
    return good_configs, bad_configs

In [9]:
good_configs = {}
bad_configs = {}

### For Pegasus Wikihow 

In [9]:
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

model_name = "google/pegasus-wikihow"
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=3)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

PyTorch version 1.6.0 available.
TensorFlow version 2.3.1 available.



good configs: 2, best config: 0


### For Pegasus: reddit_tifu Window Size 1 - 7 

In [11]:
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

model_name = "google/pegasus-reddit_tifu"
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size = 3)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

No good config found. best config so far reached: 3 with mean: 0.43972091576384875 and comp: 0.5906845161292759


### For Pegasus cnn-dailymail:  Window Size 1-7

In [9]:
model_name = "google/pegasus-cnn_dailymail"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size = 1)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

PyTorch version 1.6.0 available.
TensorFlow version 2.3.1 available.



good configs: 2, best config: 0


### For Pegasus xsum:  Window Size 1-7

In [10]:
model_name = "google/pegasus-xsum"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=1)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))


No good config found. best config so far reached: 7 with mean: 0.4141865272891903 and comp: 0.5770692673127056


### For Pegasus Large: Window Size 1-7 

In [11]:
model_name = "google/pegasus-large"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=1)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))


good configs: 2, best config: 0


### For Pegasus Newsroom

In [9]:
model_name = "google/pegasus-newsroom"

good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=2)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

PyTorch version 1.6.0 available.
TensorFlow version 2.3.1 available.



good configs: 3, best config: 1


### For Pegasus Multinews 

In [9]:
model_name = "google/pegasus-multi_news"

good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=1)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

PyTorch version 1.6.0 available.
TensorFlow version 2.3.1 available.



good configs: 5, best config: 4


### For Pegasus Gigaword 

In [10]:
model_name = "google/pegasus-gigaword"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=1)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))


No good config found. best config so far reached: 3 with mean: 0.1067779269274208 and comp: 0.2613798969839734


### For T5: Window Size 1 - 7 

In [11]:
model_name = "t5-large"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=1)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

/home/science/anaconda3/envs/tr_pegasus_env/lib/python3.7/site-packages/transformers/modeling_auto.py:766: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,



good configs: 2, best config: 1


### For Bart: Window-Size 1-7 

In [12]:
model_name = "facebook/bart-large-cnn"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=1)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))


good configs: 3, best config: 0


In [13]:
model_name = "facebook/bart-large-xsum"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, batch_size=1)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))


good configs: 3, best config: 1


In [14]:
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
configs = good_configs

In [85]:
import numpy as np
rating_config = {}
for model_name in configs.keys():
    i = 0
    for c in configs[model_name]:
        c_rate_mean = c["compression_rate mean"]
        c_rate_std_dev = c["compression_rate std dev"]
        q_mean = c["gen_quality mean"]
        ws = c["window_size"]
        overlap = c["overlap"]
        q_std_dev = c["gen_quality std dev"]
        score = (q_mean - 2*np.abs(0.45-c_rate_mean)) - c_rate_std_dev - q_std_dev
        #score = c_rate_std_dev
        #score = (np.abs(c_rate-q_mean) + (0.45-c_rate)**2 + (2 * (0.45-q_mean)**2)+0.5*(0-std_dev)**2)/5
        #score = (((0.45-c_rate)**2)+(1*(0.45-q_mean)**2)+0.5*(0-std_dev)**2)/3
        rating_config[f"{model_name} i: {i} ws: {ws}, ov: {overlap}"] = score
        i = i+1

### Results

In [86]:
{k:v for k,v in sorted(rating_config.items(),key=lambda v: v[1], reverse=True)}

{'facebook/bart-large-cnn i: 0 ws: 6, ov: 1': 0.4212760036215957,
 'facebook/bart-large-cnn i: 2 ws: 14, ov: 5': 0.41923443881267425,
 'google/pegasus-large i: 1 ws: 14, ov: 5': 0.3820490546332991,
 'google/pegasus-cnn_dailymail i: 1 ws: 10, ov: 5': 0.36422377425865476,
 'facebook/bart-large-cnn i: 1 ws: 10, ov: 2': 0.3635289033437857,
 'google/pegasus-wikihow i: 0 ws: 6, ov: 2': 0.3338638677832783,
 't5-large i: 0 ws: 6, ov: 1': 0.30654865826563893,
 'google/pegasus-large i: 0 ws: 6, ov: 2': 0.30166319534998015,
 'google/pegasus-cnn_dailymail i: 0 ws: 6, ov: 3': 0.27353355900666354,
 'facebook/bart-large-xsum i: 2 ws: 14, ov: 5': 0.2710880529901656,
 'facebook/bart-large-xsum i: 0 ws: 6, ov: 3': 0.261342113361323,
 'facebook/bart-large-xsum i: 1 ws: 10, ov: 5': 0.25139232270797995,
 't5-large i: 1 ws: 6, ov: 2': 0.24435922930876075,
 'google/pegasus-multi_news i: 2 ws: 22, ov: 0': 0.2283749258660765,
 'google/pegasus-wikihow i: 1 ws: 14, ov: 5': 0.21570878808401564,
 'google/pegasus-m

Results show, that the above configs do not change when shuffling the weight, which is an indicator, that the top three models indeed could perform the best

In [87]:
good_configs["facebook/bart-large-cnn"][0]

{'model_name': 'facebook/bart-large-cnn',
 'window_size': 6,
 'overlap': 1,
 'compression_rate mean': 0.4524026375034687,
 'compression_rate std dev': 0.06601373028372438,
 'gen_quality mean': 0.5540161700477096,
 'gen_quality std dev': 0.061921161135452214,
 'example_gen': 'I am a massive fan of Sherlock the TV show. Its probably my favourite TV show ever. Im also a big fan of the original Sherlock Holmes stories by Sir Arthur Conan Doyle. Therefore, A Study in Charlotte seemed like the perfect read for me. This was by no means a bad book and lots of fans of Sherlock would love this and eat it right up. For those of you going through Sherlock withdrawals, this will fill the gaping hole in your heart. It also has a wonderful mystery that references and incorporates some of the elements in Sir Arthur Conan Doyles original works that will have you dying to read the original stories. I enjoyed reading a modern, high school detective mystery that was reminiscent of a Sherlock Holmes story.

In [88]:
good_configs["google/pegasus-large"][1]

{'model_name': 'google/pegasus-large',
 'window_size': 14,
 'overlap': 5,
 'compression_rate mean': 0.43515864104774543,
 'compression_rate std dev': 0.02096309697012672,
 'gen_quality mean': 0.4733341528283593,
 'gen_quality std dev': 0.04063928332042435,
 'example_gen': 'Im also a big fan of the original Sherlock Holmes stories by Sir Arthur Conan Doyle even though Ive only read a couple of the short stories and two of the novels. It also has a wonderful mystery that references and incorporates some of the elements in Sir Arthur Conan Doyles original works that will have you dying to read the original stories. I also thought that it was wonderfully translated to a high school setting, and I enjoyed reading a modern, high school detective mystery that was reminiscent of a Sherlock Holmes story. For me, the characters of Charlotte Holmes and Jamie Watson too closely resembled Sherlock Holmes and John Watson for me to enjoy the book. For me, the characters of Charlotte Holmes and Jamie 

In [93]:
good_configs["t5-large"][0]

{'model_name': 't5-large',
 'window_size': 6,
 'overlap': 1,
 'compression_rate mean': 0.42997412916831373,
 'compression_rate std dev': 0.05748861647876677,
 'gen_quality mean': 0.46063699467869235,
 'gen_quality std dev': 0.05654797827091405,
 'example_gen': 'i am a massive fan of Sherlock the TV show. its probably my favourite TV show ever . i also love the original Sherlock Holmes stories by sir Arthur conan Doyle . a study in Charlotte seemed like the perfect read for me . A Study in Charlotte was a perfect read for me because no Sherlock fan would say no to more Sherlock or Sherlockinspired stories . sadly, this book fell a little bit short for me . this was by no means a bad book and lots of fans of Sherlock would love this . the book has a wonderful mystery that references and incorporates some of the elements in Sir Arthur conan Doyles original works . it was translated to a high school setting and i enjoyed reading a modern, high school detective mystery that was reminiscent 

Final configs are:

In [ ]:
good_configs["google/pegasus-large"][1]
good_configs["facebook/bart-large-cnn"][0]
good_configs["t5-large"][0]

# Find Window size for finetuned bart

In [10]:
model_name = "bart-custom-large_finetuned/best_tfmr/"

good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries, tokenizertf, starti=50, rangei=75, inc=5 ,batch_size=4)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate mean']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

PyTorch version 1.6.0 available.
TensorFlow version 2.3.1 available.



good configs: 6, best config: 0


In [20]:
import pickle as pkl
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
configs = good_configs
import pandas as pd

In [21]:
import numpy as np
rating_config = {}
dfs = []
good_configs.pop("bart-custom-large_finetuned/best_tfmr/")
for model_name in configs.keys():
    i = 0
    for c in configs[model_name]:
        c_rate_mean = c["compression_rate mean"]
        c_rate_std_dev = c["compression_rate std dev"]
        q_mean = c["gen_quality mean"]
        ws = c["window_size"]
        overlap = c["overlap"]
        q_std_dev = c["gen_quality std dev"]
        score = (q_mean - 2*np.abs(0.45-c_rate_mean)) - c_rate_std_dev - q_std_dev
        #score = c_rate_std_dev
        #score = (np.abs(c_rate-q_mean) + (0.45-c_rate)**2 + (2 * (0.45-q_mean)**2)+0.5*(0-std_dev)**2)/5
        #score = (((0.45-c_rate)**2)+(1*(0.45-q_mean)**2)+0.5*(0-std_dev)**2)/3
        rating_config[f"{model_name} i: {i} ws: {ws}, ov: {overlap}"] = score
        dfs.append({"model_name": model_name, "score": score, "window_size": ws, "overlap": overlap, "index": i})
        i = i+1

In [22]:
df = pd.DataFrame.from_dict(dfs)
df.sort_values(by="score",ascending=False)

,model_name,score,window_size,overlap,index
16,facebook/bart-large-cnn,0.421276,6,1,0
18,facebook/bart-large-cnn,0.419234,14,5,2
5,google/pegasus-large,0.382049,14,5,1
3,google/pegasus-cnn_dailymail,0.364224,10,5,1
17,facebook/bart-large-cnn,0.363529,10,2,1
0,google/pegasus-wikihow,0.333864,6,2,0
14,t5-large,0.306549,6,1,0
4,google/pegasus-large,0.301663,6,2,0
2,google/pegasus-cnn_dailymail,0.273534,6,3,0
21,facebook/bart-large-xsum,0.271088,14,5,2


In [30]:
configs["google/pegasus-newsroom"][0]

{'model_name': 'google/pegasus-newsroom',
 'window_size': 6,
 'overlap': 3,
 'compression_rate mean': 0.49389199664132,
 'compression_rate std dev': 0.0949510136810517,
 'gen_quality mean': 0.44243917089823703,
 'gen_quality std dev': 0.1076761295580852,
 'example_gen': 'This review also appears on Happy Indulgence. Check it out for more reviews. I am a massive fan of Sherlock the TV show. Its probably my favourite TV show ever. Im also a big fan of the original Sherlock Holmes stories by... Collection of all USATODAY.com coverage of A Study in Charlotte, including articles, videos, photos, and quotes.<n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n><n> Collection of all USATODAY.com coverage of The Abominable Bride., including articles, videos, photos, and quotes. <n> <n> <n> <n> <n>Sadly, I set my expectations a little bit too high and this book fell a little bit short for me. Browse Sherlock Holmes la

In [6]:
{k:v for k,v in sorted(rating_config.items(),key=lambda v: v[1], reverse=True)}

{'facebook/bart-large-cnn i: 0 ws: 6, ov: 1': 0.4212760036215957,
 'facebook/bart-large-cnn i: 2 ws: 14, ov: 5': 0.41923443881267425,
 'google/pegasus-large i: 1 ws: 14, ov: 5': 0.3820490546332991,
 'google/pegasus-cnn_dailymail i: 1 ws: 10, ov: 5': 0.36422377425865476,
 'facebook/bart-large-cnn i: 1 ws: 10, ov: 2': 0.3635289033437857,
 'google/pegasus-wikihow i: 0 ws: 6, ov: 2': 0.3338638677832783,
 't5-large i: 0 ws: 6, ov: 1': 0.30654865826563893,
 'google/pegasus-large i: 0 ws: 6, ov: 2': 0.30166319534998015,
 'google/pegasus-cnn_dailymail i: 0 ws: 6, ov: 3': 0.27353355900666354,
 'facebook/bart-large-xsum i: 2 ws: 14, ov: 5': 0.2710880529901656,
 'facebook/bart-large-xsum i: 0 ws: 6, ov: 3': 0.261342113361323,
 'facebook/bart-large-xsum i: 1 ws: 10, ov: 5': 0.25139232270797995,
 't5-large i: 1 ws: 6, ov: 2': 0.24435922930876075,
 'google/pegasus-multi_news i: 2 ws: 22, ov: 0': 0.2283749258660765,
 'google/pegasus-wikihow i: 1 ws: 14, ov: 5': 0.21570878808401564,
 'google/pegasus-m

In [8]:
(0.46424840215718943 - 2*np.abs(0.45-0.5359814907931674)) - 0.16365722708560757 - 0.09852072828764007

0.030107465197607083

In [11]:
good_configs["bart-custom-large_finetuned/best_tfmr/"][2]

{'model_name': 'bart-custom-large_finetuned/best_tfmr/',
 'window_size': 70,
 'overlap': 0,
 'compression_rate mean': 0.5060919855059665,
 'compression_rate std dev': 0.16276220027249935,
 'gen_quality mean': 0.4395493510261279,
 'gen_quality std dev': 0.09145904162061354,
 'example_gen': ' A Study in Charlotte is a retelling of the Sherlock Holmes stories by Sir Arthur Conan Doyle. Ive read the original Sherlock Holmes books, but Ive never read any of the other Holmes stories, so I had no idea what to expect from this retelling. I was so excited to read this book because Ive heard so many great things about it. I have to admit, I was a little disappointed with the quality of this book. I really enjoyed the original stories, but this book was not as good as I expected it to be. The characters were all very similar to the original Holmes stories. I found Charlotte and Jamie to be very similar. Their personalities were very similar and I found it hard to connect with either of them. I fe